In [ ]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator

In [ ]:
@dlt.resource(name="ny_taxi", write_disposition="append")
def ny_taxi(cursor_date = dlt.sources.incremental("Trip_Dropoff_DateTime", initial_value="2009-06-15")):

    client = RESTClient(
        base_url="https://us-central1-dlthub-analytics.cloudfunctions.net", 
        paginator=PageNumberPaginator(
            base_page=1, 
            total_path=None
        )
    )

    for page in client.paginate("data_engineering_zoomcamp_api"):
        yield page


pipeline = dlt.pipeline(
    pipeline_name="ny_taxi_incremental", 
    destination="duckdb", 
    dataset_name="ny_taxi_data"
)

load_info = pipeline.run(ny_taxi)
print(load_info)
print(pipeline.last_trace)

In [ ]:
with pipeline.sql_client() as client: 
    result = client.execute_sql(
        """
        SELECT 
        MIN(trip_dropoff_date_time)
        FROM ny_taxi;
        """
    )

    print(result)

In [ ]:
# Run the same pipeline again

pipeline = dlt.pipeline(
    pipeline_name="ny_taxi_incremental", 
    destination="duckdb", 
    dataset_name="ny_taxi_data"
)

load_info = pipeline.run(ny_taxi)
print(load_info)
print(pipeline.last_trace)